# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [3]:
# import libraries here
import json
import requests
import pandas as pd
from pandas import json_normalize

Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [25]:
# Lire le fichier du token 
token = '470093b7ef5be6868143c56ebfd6cc793044cb0c'

In [28]:
# your code here
h = {'Authorization':'470093b7ef5be6868143c56ebfd6cc793044cb0c'}
res = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks', headers=h)


In [29]:

forks

[{'id': 276714038,
  'node_id': 'MDEwOlJlcG9zaXRvcnkyNzY3MTQwMzg=',
  'name': 'dataV2-labs',
  'full_name': 'cmerticaru/dataV2-labs',
  'private': False,
  'owner': {'login': 'cmerticaru',
   'id': 61248268,
   'node_id': 'MDQ6VXNlcjYxMjQ4MjY4',
   'avatar_url': 'https://avatars1.githubusercontent.com/u/61248268?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/cmerticaru',
   'html_url': 'https://github.com/cmerticaru',
   'followers_url': 'https://api.github.com/users/cmerticaru/followers',
   'following_url': 'https://api.github.com/users/cmerticaru/following{/other_user}',
   'gists_url': 'https://api.github.com/users/cmerticaru/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/cmerticaru/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/cmerticaru/subscriptions',
   'organizations_url': 'https://api.github.com/users/cmerticaru/orgs',
   'repos_url': 'https://api.github.com/users/cmerticaru/repos',
   'events_url': 'htt

In [31]:
# Flatten data
flattened_data = json_normalize(forks)
flattened_data

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,owner.following_url,owner.gists_url,owner.starred_url,owner.subscriptions_url,owner.organizations_url,owner.repos_url,owner.events_url,owner.received_events_url,owner.type,owner.site_admin
0,276714038,MDEwOlJlcG9zaXRvcnkyNzY3MTQwMzg=,dataV2-labs,cmerticaru/dataV2-labs,False,https://github.com/cmerticaru/dataV2-labs,None,True,https://api.github.com/repos/cmerticaru/dataV2...,https://api.github.com/repos/cmerticaru/dataV2...,...,https://api.github.com/users/cmerticaru/follow...,https://api.github.com/users/cmerticaru/gists{...,https://api.github.com/users/cmerticaru/starre...,https://api.github.com/users/cmerticaru/subscr...,https://api.github.com/users/cmerticaru/orgs,https://api.github.com/users/cmerticaru/repos,https://api.github.com/users/cmerticaru/events...,https://api.github.com/users/cmerticaru/receiv...,User,False
1,274415901,MDEwOlJlcG9zaXRvcnkyNzQ0MTU5MDE=,dataV2-labs,aaronepv/dataV2-labs,False,https://github.com/aaronepv/dataV2-labs,None,True,https://api.github.com/repos/aaronepv/dataV2-labs,https://api.github.com/repos/aaronepv/dataV2-l...,...,https://api.github.com/users/aaronepv/followin...,https://api.github.com/users/aaronepv/gists{/g...,https://api.github.com/users/aaronepv/starred{...,https://api.github.com/users/aaronepv/subscrip...,https://api.github.com/users/aaronepv/orgs,https://api.github.com/users/aaronepv/repos,https://api.github.com/users/aaronepv/events{/...,https://api.github.com/users/aaronepv/received...,User,False
2,274151443,MDEwOlJlcG9zaXRvcnkyNzQxNTE0NDM=,dataV2-labs,Sanka101/dataV2-labs,False,https://github.com/Sanka101/dataV2-labs,None,True,https://api.github.com/repos/Sanka101/dataV2-labs,https://api.github.com/repos/Sanka101/dataV2-l...,...,https://api.github.com/users/Sanka101/followin...,https://api.github.com/users/Sanka101/gists{/g...,https://api.github.com/users/Sanka101/starred{...,https://api.github.com/users/Sanka101/subscrip...,https://api.github.com/users/Sanka101/orgs,https://api.github.com/users/Sanka101/repos,https://api.github.com/users/Sanka101/events{/...,https://api.github.com/users/Sanka101/received...,User,False
3,272914619,MDEwOlJlcG9zaXRvcnkyNzI5MTQ2MTk=,dataV2-labs,Lucille-Art/dataV2-labs,False,https://github.com/Lucille-Art/dataV2-labs,None,True,https://api.github.com/repos/Lucille-Art/dataV...,https://api.github.com/repos/Lucille-Art/dataV...,...,https://api.github.com/users/Lucille-Art/follo...,https://api.github.com/users/Lucille-Art/gists...,https://api.github.com/users/Lucille-Art/starr...,https://api.github.com/users/Lucille-Art/subsc...,https://api.github.com/users/Lucille-Art/orgs,https://api.github.com/users/Lucille-Art/repos,https://api.github.com/users/Lucille-Art/event...,https://api.github.com/users/Lucille-Art/recei...,User,False
4,272536402,MDEwOlJlcG9zaXRvcnkyNzI1MzY0MDI=,dataV2-labs,Mast3rSensei/dataV2-labs,False,https://github.com/Mast3rSensei/dataV2-labs,None,True,https://api.github.com/repos/Mast3rSensei/data...,https://api.github.com/repos/Mast3rSensei/data...,...,https://api.github.com/users/Mast3rSensei/foll...,https://api.github.com/users/Mast3rSensei/gist...,https://api.github.com/users/Mast3rSensei/star...,https://api.github.com/users/Mast3rSensei/subs...,https://api.github.com/users/Mast3rSensei/orgs,https://api.github.com/users/Mast3rSensei/repos,https://api.github.com/users/Mast3rSensei/even...,https://api.github.com/users/Mast3rSensei/rece...,User,False
5,271771152,MDEwOlJlcG9zaXRvcnkyNzE3NzExNTI=,dataV2-labs,guillemperdigo/dataV2-labs,False,https://github.com/guillemperdigo/dataV2-labs,None,True,https://api.github.com/repos/guillemperdigo/da...,https://api.github.com/repos/guillemperdigo/da...,...,https://api.github.com/users/guillemperdigo/fo...,https://api.github.com/users/guillemperdigo/gi...,https://api.github.com/users/guillemperdigo/st...,https://api.github.com/users/guillemperdigo/su...,https://api.github.com/users/guillemperdigo/orgs,https://api.githu

#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
Print the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [ ]:
df.

In [42]:
# Loop Json response
total_data = range(30)

df=pd.DataFrame()
for i in total_data:
    res = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks', headers=h)
    forks = res.json()
    flattened_data = json_normalize(forks)
    flattened_data = flattened_data.set_index('language')
    df = df.append(flattened_data)

df

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,owner.following_url,owner.gists_url,owner.starred_url,owner.subscriptions_url,owner.organizations_url,owner.repos_url,owner.events_url,owner.received_events_url,owner.type,owner.site_admin
language,,,,,,,,,,,,,,,,,,,,,
NaN,276714038,MDEwOlJlcG9zaXRvcnkyNzY3MTQwMzg=,dataV2-labs,cmerticaru/dataV2-labs,False,https://github.com/cmerticaru/dataV2-labs,None,True,https://api.github.com/repos/cmerticaru/dataV2...,https://api.github.com/repos/cmerticaru/dataV2...,...,https://api.github.com/users/cmerticaru/follow...,https://api.github.com/users/cmerticaru/gists{...,https://api.github.com/users/cmerticaru/starre...,https://api.github.com/users/cmerticaru/subscr...,https://api.github.com/users/cmerticaru/orgs,https://api.github.com/users/cmerticaru/repos,https://api.github.com/users/cmerticaru/events...,https://api.github.com/users/cmerticaru/receiv...,User,False
NaN,274415901,MDEwOlJlcG9zaXRvcnkyNzQ0MTU5MDE=,dataV2-labs,aaronepv/dataV2-labs,False,https://github.com/aaronepv/dataV2-labs,None,True,https://api.github.com/repos/aaronepv/dataV2-labs,https://api.github.com/repos/aaronepv/dataV2-l...,...,https://api.github.com/users/aaronepv/followin...,https://api.github.com/users/aaronepv/gists{/g...,https://api.github.com/users/aaronepv/starred{...,https://api.github.com/users/aaronepv/subscrip...,https://api.github.com/users/aaronepv/orgs,https://api.github.com/users/aaronepv/repos,https://api.github.com/users/aaronepv/events{/...,https://api.github.com/users/aaronepv/received...,User,False
NaN,274151443,MDEwOlJlcG9zaXRvcnkyNzQxNTE0NDM=,dataV2-labs,Sanka101/dataV2-labs,False,https://github.com/Sanka101/dataV2-labs,None,True,https://api.github.com/repos/Sanka101/dataV2-labs,https://api.github.com/repos/Sanka101/dataV2-l...,...,https://api.github.com/users/Sanka101/followin...,https://api.github.com/users/Sanka101/gists{/g...,https://api.github.com/users/Sanka101/starred{...,https://api.github.com/users/Sanka101/subscrip...,https://api.github.com/users/Sanka101/orgs,https://api.github.com/users/Sanka101/repos,https://api.github.com/users/Sanka101/events{/...,https://api.github.com/users/Sanka101/received...,User,False
NaN,272914619,MDEwOlJlcG9zaXRvcnkyNzI5MTQ2MTk=,dataV2-labs,Lucille-Art/dataV2-labs,False,https://github.com/Lucille-Art/dataV2-labs,None,True,https://api.github.com/repos/Lucille-Art/dataV...,https://api.github.com/repos/Lucille-Art/dataV...,...,https://api.github.com/users/Lucille-Art/follo...,https://api.github.com/users/Lucille-Art/gists...,https://api.github.com/users/Lucille-Art/starr...,https://api.github.com/users/Lucille-Art/subsc...,https://api.github.com/users/Lucille-Art/orgs,https://api.github.com/users/Lucille-Art/repos,https://api.github.com/users/Lucille-Art/event...,https://api.github.com/users/Lucille-Art/recei...,User,False
NaN,272536402,MDEwOlJlcG9zaXRvcnkyNzI1MzY0MDI=,dataV2-labs,Mast3rSensei/dataV2-labs,False,https://github.com/Mast3rSensei/dataV2-labs,None,True,https://api.github.com/repos/Mast3rSensei/data...,https://api.github.com/repos/Mast3rSensei/data...,...,https://api.github.com/users/Mast3rSensei/foll...,https://api.github.com/users/Mast3rSensei/gist...,https://api.github.com/users/Mast3rSensei/star...,https://api.github.com/users/Mast3rSensei/subs...,https://api.github.com/users/Mast3rSensei/orgs,https://api.github.com/users/Mast3rSensei/repos,https://api.github.com/users/Mast3rSensei/even...,https://api.github.com/users/Mast3rSensei/rece...,User,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jupyter Notebook,261143673,MDEwOlJlcG9zaXRvcnkyNjExNDM2NzM=,dataV2-labs,FergusCuthill/dataV2-labs,False,https://github.com/FergusCuthill/dataV2-labs,None,True,https://api.github.com/repos/FergusCuthill/dat...,https://api.github.com/repos/FergusCuthill/dat...,...,https://api.github.com/users/FergusCuthill/fol...,https://api.github.com/users/FergusCuthill/gis...,https://api.github.com/us

In [49]:
df.index.value_counts()

Jupyter Notebook    630
Name: language, dtype: int64

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [62]:
# your code here
total_data = range(30)

df=pd.DataFrame()
for i in total_data:
    res = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks', headers=h)
    forks = res.json()
    flattened_data = json_normalize(forks)
    flattened_data = flattened_data.set_index('updated_at')
    'updated_at' =='2019-10'
    
    df1 = df.append(flattened_data)

df1
#.value_counts()

KeyError: "None of ['updated_at'] are in the columns"

#### 2. Count how many commit objects are contained in the array.

In [ ]:
results = ""
n_page = 0
nb_results = 100
url = f"https://api.github.com/repos/ironha-datalabs/dataV2-labs/commits?since=2019-10-01T00:00:00&until=2019-30-31T23:59:599"

while True:
    n_page += 1
    
    response = resquests.get(f"{url}&per_page={nb_results}&page={n_page}",headers = {"Ahthorization":"token"})
    results = response.json()
    
    if results == "":
        results = result
    else:
        results+= result
    
    print(f"page : {n_page} - résultats : {len(result)}")
    
    if len(result) != n

In [ ]:
# your code here
len(results)

## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [ ]:
# your code here

#### 2.  Sort the filenames ascendingly.

In [ ]:
# your code here

#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [ ]:
# your code here

#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [ ]:
# your code here

#### 5. Print out the joke.

In [ ]:
# your code here